In [ ]:
# General python imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!pip install cobaya

# Attention:

Keep always the Cobaya documentation open: https://cobaya.readthedocs.io

# Exercise 1:

### Use the Metropolis-Hastings MCMC sampler of Cobaya to sample a simple posterior distribution

In this exercise, we aim to sample the posterior distribution corresponding to the product of a gaussian mixture likelihood in Cobaya (see https://cobaya.readthedocs.io/en/latest/likelihood_gaussian_mixture.html) using simple priors for the parameters.

For that, you need to fill up the blocks `likelihood`, `params` and `sampler` of the configuration dictionary and then run Cobaya.

In [ ]:
# Fill the configuration dictionary called 'info' below

info = {
# We import the likelihood from Cobaya likelihood libraries
#'likelihood': Cobaya's protected key of the input dictionary:
    "likelihood": {
        "gaussian_mixture": {
            "means": [0.2, 0],
            "covs": [[0.1, 0.05],
                     [0.05, 0.2]],
            "derived": True}},
#'params': Cobaya's protected key of the input dictionary.
# Includes the parameters that the user would like to sample over:
    "params": dict([
        ("a", {
            "prior": {"min": -0.5, "max": 3},
            "latex": r"\alpha"}),
        ("b", {
            "prior": {"dist": "norm", "loc": 0, "scale": 1},
            "ref": 0,
            "proposal": 0.5,
            "latex": r"\beta"}),
        ("hyperparam_a", {
            "latex": r"\alpha^\prime"}),
        ("derived_b", {
            "latex": r"\beta^\prime"})]),
# 'sampler': Cobaya's protected key of the input dictionary.
# you can choose the sampler you want to use.
# Check Cobaya's documentation to see the list of available samplers
    "sampler": {
        "mcmc": None},
# 'force': Cobaya's protected key of the input dictionary.
# 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    "force": True,
# 'debug': Cobaya's protected key of the input dictionary.
# how much information you want Cobaya to print? If debug: True, it prints every single detail
# that is going on internally in Cobaya
    "debug": False,
# 'output': Cobaya's protected key of the input dictionary.
# Where are the results going to be stored, in case that the sampler produce output files?
# For example: chains...
# modify the path below within 'output' to choose a name and a directory for those files
    "output": "chains_exercise1"}

In [ ]:
#  We import Cobaya
from cobaya.run import run
# Now we run!
updated_info, sampler = run(info)

### Plot the corresponding distribution using GetDist

Do not forget to open the GetDist html DEMO: https://getdist.readthedocs.io/en/latest/plot_gallery.html

In [ ]:
# Import corresponding modules from GetDist
from getdist.mcsamples import loadMCSamples, MCSamplesFromCobaya
import getdist.plots as gdplt

# Load the chains and the updated config file from the previous run
gd_sample = MCSamplesFromCobaya(updated_info, sampler.products()["sample"])

# Analyze the chains (print the mean and covmat according to GetDist documentation)
mean = gd_sample.getMeans()[:2]
covmat = gd_sample.getCovMat().matrix[:2, :2]
print("Mean:")
print(mean)
print("Covariance matrix:")
print(covmat)

# PLOT THE CHAINS (have a look at the DEMO above from GetDist)
# The goal is to produce a triangle plot

plt.figure()
gdplot = gdplt.get_subplot_plotter()
gdplot.triangle_plot(gd_sample, ["a", "b"], filled=True)
plt.show()

### Now, let's change the likelihood above by a new gaussian ring likelihood defined on cartesian coordinates

Cobaya allows to pass self-defined likelihoods as likelihoods of the `likelihood` block.

In [ ]:
# We define the new likelihood as a python function
# ATTENTION: Cobaya always needs to return the loglike!

def gauss_ring_logp(x, y, mean_radius=1, std=0.02):
    """
    Defines a gaussian ring likelihood on cartesian coordinater,
    around some ``mean_radius`` and with some ``std``.
    """
    return stats.norm.logpdf(np.sqrt(x**2 + y**2), loc=mean_radius, scale=std)

In [ ]:
# Let's update the new likelihood in the info dictionary above

info = {"likelihood": {"ring": gauss_ring_logp}}

In [ ]:
# The new likelihood requests two new parameters: x and y
# Therefore, we need to define them in the params block of the info dictionary above adding a new prior

info["params"] = {
    "x": {"prior": {"min": 0, "max": 2}, "ref": 0.5, "proposal": 0.01},
    "y": {"prior": {"min": 0, "max": 2}, "ref": 0.5, "proposal": 0.01}}

In [ ]:
# We add the polar coordinates theta and radius as derived parameters within the params block

def get_r(x, y):
    return np.sqrt(x ** 2 + y ** 2)


def get_theta(x, y):
    return np.arctan(y / x)

info["params"]["r"] = {"derived": get_r}
info["params"]["theta"] = {"derived": get_theta,
                           "latex": r"\theta", "min": 0, "max": np.pi/2}

In [ ]:
# We add some new conditions in the sampler block, to give extra tries in case the mcmc gets stuck

info["sampler"] = {"mcmc": {"Rminus1_stop": 0.001, "max_tries": 1000}}

# Re-run cobaya as we did above!
updated_info, sampler = run(info)

### Replot the chains using GetDist

We aim to create two triangle plots: one for the cartesian coordinates x and y, and another for the polar coordinates theta and radius

In [ ]:
# Load the chains and the updated config file from the previous run
# Make two triangle plots: one for the cartesian coordinates x and y,
# and a different triangle plot for the radius and angle theta

# Load the chains and the updated config file from the previous run
gdsamples = MCSamplesFromCobaya(updated_info, sampler.products()["sample"])

gdplot_cartesian = gdplt.get_subplot_plotter(width_inch=5)
gdplot_cartesian.triangle_plot(gdsamples, ["x", "y"], filled=True)
plt.suptitle('Cartesian',  va='bottom')
gdplot_polar = gdplt.get_subplot_plotter(width_inch=5)
gdplot_polar.triangle_plot(gdsamples, ["r", "theta"], filled=True)
plt.suptitle('Polar', va='bottom');
plt.show()

# Exercise 2:

### Make an evaluation of the full posterior of the parameters of $\Lambda$CDM using BAO + SN + PLANCK18

In this exercise, we aim to make an evaluation of the posterior distribution of the Standard Cosmological Model parameters using BAO + SN + PLANCK18 data.

For that, you need to create a new config file speciying the corresponding likelihoods, the parameters of the $\Lambda$CDM model and specify the sampler to `evaluate`.

What about the `theory` module? Do you need to specify it? What would you choose?

If you feel like working, you can change the sampler `evaluate` to the sampler `mcmc` to explore the full posterior distribution. Attention: it will take around 3 or 4 days. Let it run for one night so that tomorrow you can have a look at the chains.

- Check this link to see how you can define the new corresponding config file: https://cobaya.readthedocs.io/en/latest/cosmo_basic_runs.html
- All the corresponding likelihood names are within the section “COSMOLOGICAL LIKELIHOODS” at https://cobaya.readthedocs.io/en/latest/

In [ ]:
!cobaya-install cosmo -p .

In [ ]:
# You need to start by defining a new config python dictionary
# (or if you feel brave, you can write a script like a yaml format
# and dump it as a python dictionary)

# LCDM has 6 parameters, so you need to define at least 6 parameters
# to be sampled from within the `params` block

# We start by defining a new config python dictionary called `info_cosmo`

info_cosmo = {
'likelihood': {'bao.sdss_dr16_baoplus_elg': None,
                'bao.sdss_dr16_baoplus_lrg': None,
                'bao.sdss_dr16_baoplus_lyauto': None,
                'bao.sdss_dr16_baoplus_lyxqso': None,
                'bao.sdss_dr16_baoplus_qso': None,
                'bao.sdss_dr7_mgs': None,
                'bao.sixdf_2011_bao': None,
                'planck_NPIPE_highl_CamSpec.TTTEEE.TTTEEE': None,
                'planckpr4lensing': None,
                'planck_2018_lowl.EE': None,
                'planck_2018_lowl.TT': None,
                'sn.pantheon': None},
'params': {'A': {'derived': 'lambda As: 1e9*As',
                  'latex': '10^9 A_\\mathrm{s}'},
            'As': {'latex': 'A_\\mathrm{s}',
                   'value': 'lambda logA: 1e-10*np.exp(logA)'},
            'DHBBN': {'derived': 'lambda DH: 10**5*DH',
                      'latex': '10^5 \\mathrm{D}/\\mathrm{H}'},
            'H0': {'latex': 'H_0', 'max': 100, 'min': 20},
            'YHe': {'latex': 'Y_\\mathrm{P}'},
            'Y_p': {'latex': 'Y_P^\\mathrm{BBN}'},
            'age': {'latex': '{\\rm{Age}}/\\mathrm{Gyr}'},
            'clamp': {'derived': 'lambda As, tau: 1e9*As*np.exp(-2*tau)',
                      'latex': '10^9 A_\\mathrm{s} e^{-2\\tau}'},
            'cosmomc_theta': {'derived': False,
                              'value': 'lambda theta_MC_100: '
                                       '1.e-2*theta_MC_100'},
            'logA': {'drop': True,
                     'latex': '\\log(10^{10} A_\\mathrm{s})',
                     'prior': {'max': 3.91, 'min': 1.61},
                     'proposal': 0.001,
                     'ref': {'dist': 'norm', 'loc': 3.05, 'scale': 0.001}},
            'mnu': 0.06,
            'ns': {'latex': 'n_\\mathrm{s}',
                   'prior': {'max': 1.2, 'min': 0.8},
                   'proposal': 0.002,
                   'ref': {'dist': 'norm', 'loc': 0.965, 'scale': 0.004}},
            'ombh2': {'latex': '\\Omega_\\mathrm{b} h^2',
                      'prior': {'max': 0.1, 'min': 0.005},
                      'proposal': 0.0001,
                      'ref': {'dist': 'norm', 'loc': 0.0224, 'scale': 0.0001}},
            'omch2': {'latex': '\\Omega_\\mathrm{c} h^2',
                      'prior': {'max': 0.99, 'min': 0.001},
                      'proposal': 0.0005,
                      'ref': {'dist': 'norm', 'loc': 0.12, 'scale': 0.001}},
            'omega_de': {'latex': '\\Omega_\\Lambda'},
            'omegam': {'latex': '\\Omega_\\mathrm{m}'},
            'omegamh2': {'derived': 'lambda omegam, H0: omegam*(H0/100)**2',
                         'latex': '\\Omega_\\mathrm{m} h^2'},
            'rdrag': {'latex': 'r_\\mathrm{drag}'},
            's8h5': {'derived': 'lambda sigma8, H0: sigma8*(H0*1e-2)**(-0.5)',
                     'latex': '\\sigma_8/h^{0.5}'},
            's8omegamp25': {'derived': 'lambda sigma8, omegam: '
                                       'sigma8*omegam**0.25',
                            'latex': '\\sigma_8 \\Omega_\\mathrm{m}^{0.25}'},
            's8omegamp5': {'derived': 'lambda sigma8, omegam: '
                                      'sigma8*omegam**0.5',
                           'latex': '\\sigma_8 \\Omega_\\mathrm{m}^{0.5}'},
            'sigma8': {'latex': '\\sigma_8'},
            'tau': {'latex': '\\tau_\\mathrm{reio}',
                    'prior': {'max': 0.8, 'min': 0.01},
                    'proposal': 0.003,
                    'ref': {'dist': 'norm', 'loc': 0.055, 'scale': 0.006}},
            'theta_MC_100': {'drop': True,
                             'latex': '100\\theta_\\mathrm{MC}',
                             'prior': {'max': 10, 'min': 0.5},
                             'proposal': 0.0002,
                             'ref': {'dist': 'norm',
                                     'loc': 1.04109,
                                     'scale': 0.0004},
                             'renames': 'theta'},
            'zrei': {'latex': 'z_\\mathrm{re}'}},
'sampler': {'evaluate': None},
'theory': {'camb': {'extra_args': {'bbn_predictor': 'PArthENoPE_880.2_standard.dat',
                                    'halofit_version': 'mead',
                                    'lens_potential_accuracy': 1,
                                    'nnu': 3.046,
                                    'num_massive_neutrinos': 1,
                                    'theta_H0_range': [20, 100]}}}}

In [ ]:
# Run Cobaya!
updated_info_cosmo, sampler_cosmo = run(info_cosmo)